In [1]:
import bs4
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
TOKENS = {'89b638c0524d4d688b56ff77add5f79c':'Tom',
          'bc2862f420844334a08e36b47119df75':'Sammy',
          'baef0dbf8c8f4d978ef962ea9a0277f3':'Chris'}
master=[]
for z in TOKENS.keys():
    s=Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    driver.implicitly_wait(100)
    driver.get("https://www.17lands.com/user_history/{}".format(z))
    sleep(10)
    html = driver.execute_script("return document.getElementsByTagName('tbody')[0].innerHTML")
    soup = BeautifulSoup(html, "html.parser")
    for x in range(len(soup.find_all('tr'))):
        tr= soup.find_all('tr')[x] #tr is the tag the deliniates distinct drafts
        td=tr.find_all('td') #td gives us the individual parts. because td has type Resultset, we can't find_all('a') and have to use regex
        data = [x.string for x in td] #if one of the elements has multiple tags or no obvious string, it returns None
        if 'PremierDraft' not in data:
            continue
        if len(data) != 9:
            continue
        data=list(filter(None, data)) #remove Nones from the list
        #data.pop(3) #removes Format
        data.insert(3,data[2][-1]) #adds losses after record
        data[2]=data[2][0] #converts record into wins
        if len(data) != 7:
            continue
        try:
            data.append(re.search(pattern='title="(.*?)"', string=str(td[3])).group(1)) #finds colors and appends it to the list
        except:
            continue
        links = re.findall(pattern='<a href="(.*?)</a>', string=str(td[-1])) #finds each link string
        links = ['https://www.17lands.com/'+links[x] for x in range(len(links))] #completes the link
        links = [links[x].split('>') for x in range(len(links))] #splits the link from the description
        for x in range(len(links)):
                       links[x]= [links[x][1], links[x][0]] #inverts the order of the description and link to prep for being a dictionary
        links_dict = dict(links) #turns list of lists into dictionary with descriptions as keys and links as values
        data.append(links_dict) #adds the dictionary to the list
        data.append(TOKENS[z])
        master.append(data)
    driver.close()
df = pd.DataFrame(master, columns = ['Date','Set','Wins','Losses','Format','Start Rank', 'End Rank', 'Colors', 'Links', 'Pilot'])
df['Draft'] = [df['Links'][x]['Draft'] if 'Draft' in df['Links'][x].keys() else None for x in range(len(df))]
df['Pool'] = [df['Links'][x]['Pool'] if 'Pool' in df['Links'][x].keys() else None for x in range(len(df))]
df['Details'] = [df['Links'][x]['Details'] if 'Details' in df['Links'][x].keys() else None for x in range(len(df))]
df['Deck 1'] = [df['Links'][x]['Deck 1'] if 'Deck 1' in df['Links'][x].keys() else None for x in range(len(df))]
df['Deck 2'] = [df['Links'][x]['Deck 2'] if 'Deck 2' in df['Links'][x].keys() else None for x in range(len(df))]
df['Deck 3'] = [df['Links'][x]['Deck 3'] if 'Deck 3' in df['Links'][x].keys() else None for x in range(len(df))]
df['Deck 4'] = [df['Links'][x]['Deck 4'] if 'Deck 4' in df['Links'][x].keys() else None for x in range(len(df))]
df['Deck 5'] = [df['Links'][x]['Deck 5'] if 'Deck 5' in df['Links'][x].keys() else None for x in range(len(df))]
df.drop(columns=['Links'], inplace=True)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/tommadsen/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/tommadsen/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/tommadsen/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [5]:
df

,Date,Set,Wins,Losses,Format,Start Rank,End Rank,Colors,Pilot,Draft,Pool,Details,Deck 1,Deck 2,Deck 3,Deck 4,Deck 5
0,2021-11-14 13:50,VOW,3,3,PremierDraft,Gold-3,Gold-2,WBg,Tom,https://www.17lands.com//draft/f91855643194481...,https://www.17lands.com//pool/f918556431944813...,https://www.17lands.com//details/f918556431944...,https://www.17lands.com//deck/f918556431944813...,None,None,None,None
1,2021-10-25 21:39,MID,6,3,PremierDraft,Diamond-4,Diamond-4,UR,Tom,https://www.17lands.com//draft/e961aa69b52d402...,https://www.17lands.com//pool/e961aa69b52d4020...,https://www.17lands.com//details/e961aa69b52d4...,https://www.17lands.com//deck/e961aa69b52d4020...,None,None,None,None
2,2021-10-24 17:51,MID,2,1,PremierDraft,Platinum-1,Diamond-4,UR,Tom,https://www.17lands.com//draft/4e8709a91ff1408...,https://www.17lands.com//pool/4e8709a91ff1408f...,https://www.17lands.com//details/4e8709a91ff14...,https://www.17lands.com//deck/4e8709a91ff1408f...,None,None,None,None
3,2021-10-24 14:50,MID,5,3,PremierDraft,Platinum-1,Platinum-1,UR,Tom,https://www.17lands.com//draft/acf4571cc26d44d...,https://www.17lands.com//pool/acf4571cc26d44dc...,https://www.17lands.com//details/acf4571cc26d4...,https://www.17lands.com//deck/acf4571cc26d44dc...,None,None,None,None
4,2021-10-24 13:56,MID,1,3,PremierDraft,Platinum-1,Platinum-1,URg,Tom,https://www.17lands.com//draft/a9b7568ac78542a...,https://www.17lands.com//pool/a9b7568ac78542a7...,https://www.17lands.com//details/a9b7568ac7854...,https://www.17lands.com//deck/a9b7568ac78542a7...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,2021-02-03 11:36,KHM,5,3,PremierDraft,Silver-4,Silver-3,RGw,Chris,https://www.17lands.com//draft/ba574b65cd214f5...,https://www.17lands.com//pool/ba574b65cd214f59...,https://www.17lands.com//details/ba574b65cd214...,https://www.17lands.com//deck/ba574b65cd214f59...,None,None,None,None
1106,2021-02-03 10:26,KHM,0,3,PremierDraft,Silver-4,Silver-4,BRg,Chris,https://www.17lands.com//draft/1b53e53dd2a2460...,https://www.17lands.com//pool/1b53e53dd2a24601...,https://www.17lands.com//details/1b53e53dd2a24...,https://www.17lands.com//deck/1b53e53dd2a24601...,None,None,None,None
1107,2021-02-03 09:19,KHM,1,3,PremierDraft,Silver-3,Silver-3,RG,Chris,https://www.17lands.com//draft/973c97cc897144c...,https://www.17lands.com//pool/973c97cc897144c0...,https://www.17lands.com//details/973c97cc89714...,https://www.17lands.com//deck/973c97cc897144c0...,None,None,None,None
1108,2021-02-03 07:34,KHM,5,3,PremierDraft,Bronze-1,Silver-3,RG,Chris,https://www.17lands.com//draft/9c8374995473476...,https://www.17lands.com//pool/9c8374995473476f...,https://www.17lands.com//details/9c83749954734...,https://www.17lands.com//deck/9c8374995473476f...,None,None,None,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        1080 non-null   object
 1   Set         1080 non-null   object
 2   Wins        1080 non-null   object
 3   Losses      1080 non-null   object
 4   Format      1080 non-null   object
 5   Start Rank  1080 non-null   object
 6   End Rank    1080 non-null   object
 7   Colors      1080 non-null   object
 8   Pilot       1080 non-null   object
 9   Draft       1077 non-null   object
 10  Pool        1080 non-null   object
 11  Details     1080 non-null   object
 12  Deck 1      1080 non-null   object
 13  Deck 2      243 non-null    object
 14  Deck 3      52 non-null     object
 15  Deck 4      11 non-null     object
 16  Deck 5      1 non-null      object
dtypes: object(17)
memory usage: 143.6+ KB


In [6]:
df.to_json()

'{"Date":{"0":"2021-11-14 13:50","1":"2021-10-25 21:39","2":"2021-10-24 17:51","3":"2021-10-24 14:50","4":"2021-10-24 13:56","5":"2021-10-24 11:43","6":"2021-10-23 11:13","7":"2021-10-23 08:32","8":"2021-10-22 19:46","9":"2021-10-11 19:37","10":"2021-10-11 12:27","11":"2021-10-10 19:14","12":"2021-10-10 18:33","13":"2021-10-03 19:23","14":"2021-08-07 12:47","15":"2021-06-03 14:50","16":"2021-06-03 13:07","17":"2021-06-03 12:24","18":"2021-06-03 11:37","19":"2021-06-03 11:34","20":"2021-06-03 11:34","21":"2021-05-21 13:28","22":"2021-05-21 09:51","23":"2021-05-21 08:58","24":"2021-05-21 08:07","25":"2021-05-20 18:24","26":"2021-05-19 17:16","27":"2021-05-18 08:49","28":"2021-05-17 19:54","29":"2021-05-17 18:56","30":"2021-05-17 09:12","31":"2021-05-17 08:23","32":"2021-05-16 20:33","33":"2021-05-15 20:53","34":"2021-05-15 15:16","35":"2021-05-12 13:05","36":"2021-05-12 07:50","37":"2021-05-12 07:04","38":"2021-05-11 20:11","39":"2021-05-11 19:10","40":"2021-05-11 15:12","41":"2021-05-11